In [1]:
import pandas as pd;

In [2]:
car_eff = pd.read_csv('car_dataset_with_imputer.csv')

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(car_eff, test_size=0.2, random_state=0)
print(X_train.shape)
print(X_test.shape)

(318, 9)
(80, 9)


In [4]:
Y_train = X_train['mpg']
X_train.drop('mpg', inplace=True, axis=1)

/home/dee/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [5]:
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor()

In [6]:
X_train_scaled = X_train.copy()
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_scaled = sc.fit_transform(X_train_scaled)

/home/dee/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/dee/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [7]:
forest_reg.fit(X_train_scaled, Y_train)

/home/dee/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [8]:
some_data = X_train.iloc[:5]
some_label = Y_train.iloc[:5]
some_data = sc.fit_transform(some_data)
print('Predictions:', forest_reg.predict(some_data))
print('Label:', list(some_label))

Predictions: [12.5  23.7  29.18 26.12 26.41]
Label: [15.0, 27.0, 34.3, 26.0, 31.6]


/home/dee/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/dee/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [9]:
import numpy as np
from sklearn.metrics import mean_squared_error
forest_predictions = forest_reg.predict(X_train_scaled)
forest_mse = mean_squared_error(Y_train, forest_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

1.2313707400174285

# Validation

In [10]:
import numpy as np
from sklearn.model_selection import cross_val_score
scores = cross_val_score(forest_reg, X_train_scaled, Y_train, scoring="neg_mean_squared_error", cv=20)
lin_rmse_scores = np.sqrt(-scores)
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation", scores.std())
display_scores(lin_rmse_scores)

Scores: [2.31980603 2.21798332 3.15789883 3.76658914 3.11286745 3.45320503
 3.05490589 4.70942738 2.7429979  2.44228249 1.90646433 2.48017892
 2.55309494 3.58277738 1.08650817 2.18059911 3.70642348 2.11266272
 2.66330747 2.6808531 ]
Mean: 2.796541653491708
Standard deviation 0.781638122973709


# Hyperparamter Tuning

In [11]:
from sklearn.model_selection import GridSearchCV
param_grid = [
{'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
{'bootstrap': [False, True], 'n_estimators': [2, 4, 6, 8, 10, 20], 'max_features': [2, 4, 6, 8], 'warm_start': [True, False]},
]
forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
scoring='neg_mean_squared_error')
grid_search.fit(X_train_scaled, Y_train)

/home/dee/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]}, {'bootstrap': [False, True], 'n_estimators': [2, 4, 6, 8, 10, 20], 'max_features': [2, 4, 6, 8], 'warm_start': [True, False]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [12]:
import numpy as np
from sklearn.model_selection import cross_val_score
scores = cross_val_score(grid_search.best_estimator_, X_train_scaled, Y_train, scoring="neg_mean_squared_error", cv=20)
lin_rmse_scores = np.sqrt(-scores)
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation", scores.std())
display_scores(lin_rmse_scores)

Scores: [2.64817567 2.22545255 3.73967683 2.60834927 2.80808348 2.741136
 3.04791988 4.55587087 2.44257261 2.92336947 1.42150778 2.33598112
 2.54182667 3.1945163  1.33143661 2.09499851 3.18386484 2.2303997
 3.24358624 2.41864356]
Mean: 2.6868683976929395
Standard deviation 0.7111086011613438


In [13]:
from sklearn.metrics import mean_squared_error
forest_predictions = grid_search.best_estimator_.predict(X_train_scaled)
forest_mse = mean_squared_error(Y_train, forest_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

1.1095440731735111

# Prediction On Test Set

In [14]:
final_model = grid_search.best_estimator_
Y_test = X_test["mpg"].copy()
X_test = X_test.drop("mpg", axis=1)
X_test_prepared = X_test.copy()
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_test_scaled = sc.fit_transform(X_test_prepared)
final_predictions = final_model.predict(X_test_scaled)
final_mse = mean_squared_error(Y_test, final_predictions)
final_rmse = np.sqrt(final_mse)
final_rmse

/home/dee/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/dee/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


2.597177614738738